In [4]:
import pandas as pd

df = pd.read_csv('file/health_result_label.csv')

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()

df['label'].unique()

array([0, 1, 2, 3, 4, 5, 6], dtype=int64)

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# fine-tuning된 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

In [6]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드
question_embeddings = torch.load('embeddings/optimized_question_embeddings.pth')

# 평가 모드로 설정
model.eval()

def get_embedding(input_question, tokenizer, model):
    # 입력 문장을 토크나이즈
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
        
    return cls_embedding.squeeze().numpy()  # numpy 배열로 반환



# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding을 텐서로 변환하고 차원 맞추기
        question_embedding_tensor = torch.tensor(question_embedding).unsqueeze(0)  # (1, 768)
        
        # input_embedding도 텐서로 변환하고 차원 맞추기
        input_embedding_tensor = torch.tensor(input_embedding).unsqueeze(0)  # (1, 768)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding_tensor, question_embedding_tensor).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity  # 유사도 반환 추가

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model)
    
    return best_answer, cosine_similarity, predicted_label  # 세 가지 값 반환


C:\Users\tlsek\AppData\Local\Temp\ipykernel_9436\3149965436.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/optimized_questi

In [7]:
# 예시 질문
input_question = "너무 목이 마른데요, 혹시 정수기 같은 건 없나요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)



예측된 레이블: 0
최고 유사도 답변: 쭉 직진하시다 보면, 나오는 갈림길에서 좌측으로 가시면 방사선실이 있습니다.
코사인 유사도: 1.0000001192092896


In [8]:
# 엑셀 파일에서 질문 데이터 읽어오기
df_test = pd.read_excel('file/test_question.xlsx')
test_questions = df_test.groupby('label')['question'].apply(list).to_dict()

# 모든 질문을 리스트로 변환하여 input_questions에 저장
input_questions = []
for questions in test_questions.values():
    input_questions.extend(questions)  # 각 레이블에 해당하는 질문을 추가

input_questions

# 모든 질문을 리스트로 변환하여 input_questions에 저장
input_questions = []
for questions in test_questions.values():
    input_questions.extend(questions)  # 각 레이블에 해당하는 질문을 추가

# 결과를 저장할 리스트 초기화
results = []

# 각 질문에 대해 챗봇 응답 호출
for input_question in input_questions:
    best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)
    
    # 결과를 리스트에 추가
    results.append({
        "question": input_question,
        "predicted_label": predicted_label,
        "best_answer": best_answer,
        "cosine_similarity": cosine_similarity
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
results_df[:10]

,question,predicted_label,best_answer,cosine_similarity
0,화상연고도 처방이 되나요?,0,안녕하세요. 수술 전에 잠시 입원실에서 환복하시고 쉬고 계시면 수술 안내 도와드릴 ...,1.000000
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,2,"예약 가능하신 날은 월요일, 수요일입니다. 시간대는 모두 가능합니다.",0.999980
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,1,네 접수 도와드리겠습니다. 앞에 보이는 태블릿에 개인정보 개입해 주시면 됩니다.,0.999996
3,"피가 계속 나는데, 저 죽는 걸까요?",5,약에 대한 부작용이 발생한 것 같습니다. 원장님께 전달해 드리겠습니다.,0.983996
4,접수 재등록해도 되나요?,1,네 잘 찾아오셨습니다. 그린 외과 의원 맞습니다. 새로운 그린 외과 의원에 오신 걸...,0.999996
5,오래된 상처 흉터도 없어질까요?,5,상담 도와드리겠습니다. 수술 일자를 정확히 알려주시기 바랍니다.,1.000000
6,여기 한의원도 같이 하나요? 침을 맡고 싶어요.,1,예약 시간이 지나면 예약과 접수는 다시 해주셔야 합니다. 접수증 작성 부탁드립니다.,0.999971
7,목 디스크 때문에 잠을 잘 수 없어요. 어떤 치료를 받을 수 있나요?,5,킥보드랑 부딪친 정확한 부위를 말씀해 주시기 바랍니다.,1.000000
8,"탈모 초기 인 것 같아요. 머리카락이 자꾸 빠지는데, 이것도 진료를 볼 수 있나요?",5,피부 사정에 따라 다 달라서 말씀드리기 어렵습니다. 증상은 사람마다 다릅니다.,1.000000
9,"저는 아니고, 제 아이가 열이 자꾸 오르다 말다 해요. 소아과도 진료 하나요?",5,기존 과거력에서 6개월 내의 변동 사항이 없었는지 입력 후 담당의와 면담하십시오.,1.000000
